In [1]:
from proton_driver import connect
with connect("proton://default:@localhost:8463/default") as conn:
  with conn.cursor() as cursor:
    cursor.execute("select 1")
    print(cursor.fetchone())

(1,)


In [2]:
# proton query
from proton_driver import client

c = client.Client(host='192.168.1.10', port=8463)

# query the stream and return in a iterator
rows = c.execute_iter(
    "SELECT TIMESTAMP, collectiveAuctionTurnover FROM Relationship ORDER BY TIMESTAMP DESC LIMIT 200"
)
for row in rows:
    print(row)
    

(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 202000.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 26574.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 513888.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 31964.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 642374.56)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 955650.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 118752.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 51646.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 146098.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 121662.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 457499.86)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 603075.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 30448.0)
(datetime.datetime(2025, 5, 28, 6, 15, 50, tzinfo=<UTC>), 4792659.040000001)
(datetime.datetime(2025, 5, 28, 6, 15, 50

In [3]:
import time
import datetime
import random

In [4]:
c = client.Client(host='192.168.1.10', port=8463)

In [5]:
# create a random stream if not exist
c.execute("""
        CREATE STREAM IF NOT EXISTS price_data(
            TIMESTAMP datetime,
            price float
        )
""")

[]

In [6]:
base_price = 100
current_price = base_price
volatility = 0.02

def generate_price():
    global current_price
    change_percent = random.uniform(-volatility, volatility)
    current_price = current_price * (1 + change_percent)
    noise = random.uniform(-0.5, 0.5)

    if current_price < 10:
        current_price = 10 + random.uniform(-0.5,0.5)
    return round(current_price, 2)

In [15]:
def write_data():
    counter = 0
    try:
        while True:
            timestamp = datetime.datetime.now()
            price = generate_price()
            c.execute( f"""
                INSERT INTO price_data (TIMESTAMP, price)
                VALUES ('{timestamp.strftime("%Y-%m-%d %H:%M:%S")}', {price})
        """)
            print(f"Inserted: {timestamp}, Price: {price}")
            counter += 1
            time.sleep(1)  # Sleep for 1 second between inserts
    
    except KeyboardInterrupt:
        print(f"Inserted {counter} records before stopping.")
    

In [16]:
write_data()

Inserted: 2025-05-28 14:21:16.761331, Price: 101.49
Inserted: 2025-05-28 14:21:17.790197, Price: 100.32
Inserted: 2025-05-28 14:21:18.800376, Price: 100.19
Inserted: 2025-05-28 14:21:19.812357, Price: 99.01
Inserted: 2025-05-28 14:21:20.823482, Price: 98.47
Inserted: 2025-05-28 14:21:21.835679, Price: 100.14
Inserted: 2025-05-28 14:21:22.850752, Price: 99.56
Inserted: 2025-05-28 14:21:23.863049, Price: 101.51
Inserted: 2025-05-28 14:21:24.872901, Price: 101.07
Inserted: 2025-05-28 14:21:25.890946, Price: 102.24
Inserted: 2025-05-28 14:21:26.901891, Price: 101.21
Inserted: 2025-05-28 14:21:27.911547, Price: 103.18
Inserted: 2025-05-28 14:21:28.932002, Price: 101.95
Inserted: 2025-05-28 14:21:29.943649, Price: 101.78
Inserted: 2025-05-28 14:21:30.957853, Price: 103.43
Inserted: 2025-05-28 14:21:31.969832, Price: 105.09
Inserted: 2025-05-28 14:21:32.979123, Price: 106.02
Inserted: 2025-05-28 14:21:33.991410, Price: 107.05
Inserted: 2025-05-28 14:21:35.002035, Price: 108.18
Inserted 19 rec

In [17]:
def test():
    rows = c.execute_iter(
        "SELECT TIMESTAMP, price FROM price_data ORDER BY TIMESTAMP DESC LIMIT 10"
    )
    for row in rows:
        print(row)